# Processing
Demonstration of Processing endpoints.

In [1]:
import utils.DemoClient as client
import jwt
import json
from owslib.csw import CatalogueServiceWeb
import os

## Initialise

### Client
We instantiate a client to interact with the platform. The client dynamically registers with the Authorisation Server to take part in UMA (User Managed Access) flows through which authorization is obtained for scoped access resources on behalf of the user.

In [2]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
base_domain = "develop.eoepca.org"
platform_domain = "auth." + base_domain
base_url = "https://" + platform_domain
demo = client.DemoClient(base_url)
demo.register_client()
demo.save_state()

State loaded from file: {'client_id': 'b0ae034e-b70b-4f49-81a2-bb9914016e1e', 'client_secret': '053c0437-1606-4b7b-8c6e-abfd3ed8ed79'}
client_id: b0ae034e-b70b-4f49-81a2-bb9914016e1e [REUSED]
Client state saved to file: state.json


### User
User authenticates and the client receives an ID Token (JWT) that represents the user.

In [3]:
#-------------------------------------------------------------------------------
# Authenticate as UserA and get ID Token
#-------------------------------------------------------------------------------
USER_NAME="eric"
USER_PASSWORD="defaultPWD"
user_id_token = demo.get_id_token(USER_NAME, USER_PASSWORD)

[Request] GET => https://auth.develop.eoepca.org/.well-known/uma2-configuration
<Response [200]>
token_endpoint: https://auth.develop.eoepca.org/oxauth/restv1/token
[Request] POST => https://auth.develop.eoepca.org/oxauth/restv1/token


## Workspace

In [4]:
workspace_url = "https://workspace-api." + base_domain
workspace_access_token = None
workspace_prefix = "develop-user-"

### Workspace: Get Details
Retrieve access information to the user workspace S3 bucket (bucket name, access key and secret access key)

In [ ]:
# Workspace - Get Details
workspace_name =  workspace_prefix + USER_NAME.lower()
response, workspace_access_token = demo.workspace_get_details(workspace_url, workspace_name, id_token=user_id_token, access_token=workspace_access_token)
workspace_details = response.json()
print(json.dumps(workspace_details["storage"], indent = 2))

### Inspect User Workspace
Using the access credentials, the user can inspect his S3 bucket.

In [6]:
# Bucket details
bucket_name = workspace_details["storage"]["credentials"]["bucketname"]
s3_access = workspace_details["storage"]["credentials"]["access"]
s3_secret = workspace_details["storage"]["credentials"]["secret"]

In [7]:
# Quick hack S3 access
import boto3

# Init S3 session for Creodias
S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)

# List bucket contents
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(' ->', obj.key)

 -> application-package/s-expression-2/s-expression_cwl.cwl
 -> application-package/s-expression/s-expression-0_0_2.cwl
 -> wf-0cd7134c-3511-11ed-8e4c-3620b8c8d343/S2B_MSIL2A_20200902T090559_N0214_R050_T34SFH_20200902T113910/S2B_MSIL2A_20200902T090559_N0214_R050_T34SFH_20200902T113910.json
 -> wf-0cd7134c-3511-11ed-8e4c-3620b8c8d343/S2B_MSIL2A_20200902T090559_N0214_R050_T34SFH_20200902T113910/ndvi.tif
 -> wf-0cd7134c-3511-11ed-8e4c-3620b8c8d343/catalog.json
 -> wf-0ea9f7be-3513-11ed-8e62-ee207e53c6ea/S2B_MSIL2A_20200902T090559_N0214_R050_T34SFH_20200902T113910/S2B_MSIL2A_20200902T090559_N0214_R050_T34SFH_20200902T113910.json
 -> wf-0ea9f7be-3513-11ed-8e62-ee207e53c6ea/S2B_MSIL2A_20200902T090559_N0214_R050_T34SFH_20200902T113910/ndvi.tif
 -> wf-0ea9f7be-3513-11ed-8e62-ee207e53c6ea/catalog.json
 -> wf-11f5cce6-35a3-11ed-b1c5-ee207e53c6ea/S2B_MSIL2A_20200902T090559_N0214_R050_T34SFH_20200902T113910/S2B_MSIL2A_20200902T090559_N0214_R050_T34SFH_20200902T113910.json
 -> wf-11f5cce6-35a3-11ed

### Upload Application Package and register it
User uploads the application package to his workspace

In [8]:
object = s3resource.Object(bucket_name, 'application-package/s-expression/s-expression-0_0_2.cwl')
result = object.put(Body=open('../data/s-expression-cwl.cwl', 'rb'))
res = result.get('ResponseMetadata')
if res.get('HTTPStatusCode') == 200:
    print('Application package uploaded successfully')
else:
    print('Application package not uploaded')

Application package uploaded successfully


User registers the Application Package through the Resource Catalog API

In [9]:
resource_url= f'{bucket_name}/application-package/s-expression/s-expression-0_0_2.cwl'
demo.workspace_register(workspace_url, workspace_name,resource_url, id_token=user_id_token, access_token=workspace_access_token)

[Request] POST => https://workspace-api.develop.eoepca.org/workspaces/develop-user-eric/register
UNEXPECTED status code: 400 for resource https://workspace-api.develop.eoepca.org/workspaces/develop-user-eric/register
[Workspace Registration] = 400 (Bad Request) () 


User inspects the registered Application Package through the Resource Catalogue API

In [10]:
headers = {
    'Authorization': 'Bearer ' + user_id_token
}
app_id= "s-expression"
workspace_endpoint = f'https://resource-catalogue.{workspace_prefix}{USER_NAME.lower()}.{base_domain}/csw'
print(f"{USER_NAME}'s workspace (catalogue): {workspace_endpoint}")
csw = CatalogueServiceWeb(workspace_endpoint,timeout=30,headers=headers)
csw.getrecords2(maxrecords=10)
print(f"{USER_NAME}'s workspace records:\n")
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')
print(f"URL for Application {app_id}:")
csw.records[app_id].references

eric's workspace (catalogue): https://resource-catalogue.develop-user-eric.develop.eoepca.org/csw
eric's workspace records:

identifier: s-expression
type: application
title: s expressions

identifier: simple-collection
type: dataset
title: simple-collection

identifier: https://demo.pygeoapi.io/stable
type: application
title: pygeoapi Demo instance - running latest stable 0.11.0 pygeoapi version

URL for Application s-expression:


[{'scheme': 'text/html',
  'url': 'https://raw.githubusercontent.com/EOEPCA/app-s-expression/main/README.md'},
 {'scheme': 'application/x-yaml',
  'url': 's3://develop-user-eric/application-package/s-expression-2/s-expression_cwl.cwl'},
 {'scheme': 'application/x-yaml',
  'url': 'https://cf2.cloudferro.com:8080/dd14602fcb6542ba958caab80261b617:develop-user-eric/application-package/s-expression-2/s-expression_cwl.cwl'},
 {'scheme': 'text/html',
  'url': 'https://github.com/EOEPCA/app-s-expression#readme'},
 {'scheme': 'text/html', 'url': 'https://github.com/EOEPCA/app-s-expression'},
 {'scheme': 'text/html',
  'url': 'https://raw.githubusercontent.com/EOEPCA/eoepca/develop/LICENSE'}]

## ADES
The ADES provides user-specific endpoints, using a URL path prefix.

In [11]:
# ADES URLs
ades_base_url = "http://ades." + base_domain
ades_wps_url = ades_base_url + "/" + USER_NAME + "/zoo"; print("ADES WPS endpoint:", ades_wps_url)
ades_proc_url = ades_base_url + "/" + USER_NAME + "/wps3"; print("ADES API Processes endpoint:", ades_proc_url)

ADES WPS endpoint: http://ades.develop.eoepca.org/eric/zoo
ADES API Processes endpoint: http://ades.develop.eoepca.org/eric/wps3


### ADES API Processes

In [12]:
ades_access_token = None
app_name = "s-expression-0_0_2"

#### ADES: List Processes
**GET {service_url}/processes**<br>
Provides a list of all processes 

In [13]:
# API Processes - List Processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

[Request] GET => http://ades.develop.eoepca.org/eric/wps3/processes
[Process List] = 200 (OK)
{'processes': [{'id': 'UndeployProcess', 'title': 'Undeploy Process', 'description': 'This method will undeploy a deployed processing service.', 'mutable': False, 'version': '1.0.0', 'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'], 'outputTransmission': ['value', 'reference'], 'links': [{'rel': 'self', 'type': 'application/json', 'title': 'Process Description', 'href': '/eric/wps3/processes/UndeployProcess'}]}, {'id': 's-expression-0_0_2', 'title': 's expressions', 'description': 'Applies s expressions to EO acquisitions', 'mutable': True, 'version': '0.0.2', 'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'], 'outputTransmission': ['value', 'reference'], 'links': [{'rel': 'self', 'type': 'application/json', 'title': 'Process Description', 'href': '/eric/wps3/processes/s-expression-0_0_2'}]}, {'id': 'GetStatus', 'title': 'Produce an updated ExecuteResponse docu

#### ADES: Deploy Application
**_Application Deployment Payload_**<br>
Use a helper script to create the POST body for the application deployment request, using the Application Package URL discovered earlier.

In [14]:
application_package_url=f"s3://{resource_url}"
os.system(f'../data/create-deploy-body-cwl.sh "{application_package_url}"')
!cat ../data/app-deploy-body-cwl.json

{
  "executionUnit": {
    "href": "s3://develop-user-eric/application-package/s-expression/s-expression-0_0_2.cwl",
    "type": "application/cwl"
  }
}


**POST {service_url}/processes**<br>
Deploys application to the ADES.<br>
Request body is json in which the input is the Application Package describing the application.<br>
The Application Package is a CWL Workflow that is typically provided as an href with content type **_application/atom+xml_** or **_application/cwl_**. The url to the application package can be either a http or s3 endpoint. In this example we will use the application package previously uploaded to Bob's workspace.

In [15]:
# API Processes - Deploy Application
#response, ades_access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-atom.json", id_token=user_id_token, access_token=ades_access_token)

response, access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-cwl.json", id_token=user_id_token, access_token=ades_access_token)

Application details read from file: ../data/app-deploy-body-cwl.json
[Request] POST => http://ades.develop.eoepca.org/eric/wps3/processes
[Deploy Response] = 201 (Created)


Check that the application has been correctly deployed

In [16]:
# Check deploy - list processes again
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

[Request] GET => http://ades.develop.eoepca.org/eric/wps3/processes
[Process List] = 200 (OK)
{'processes': [{'id': 'UndeployProcess', 'title': 'Undeploy Process', 'description': 'This method will undeploy a deployed processing service.', 'mutable': False, 'version': '1.0.0', 'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'], 'outputTransmission': ['value', 'reference'], 'links': [{'rel': 'self', 'type': 'application/json', 'title': 'Process Description', 'href': '/eric/wps3/processes/UndeployProcess'}]}, {'id': 's-expression-0_0_2', 'title': 's expressions', 'description': 'Applies s expressions to EO acquisitions', 'mutable': True, 'version': '0.0.2', 'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'], 'outputTransmission': ['value', 'reference'], 'links': [{'rel': 'self', 'type': 'application/json', 'title': 'Process Description', 'href': '/eric/wps3/processes/s-expression-0_0_2'}]}, {'id': 'GetStatus', 'title': 'Produce an updated ExecuteResponse docu

#### ADES: Get Application Details
**GET {service_url}/processes/{application_name}**<br>
Provides details of the deployed application<br>
The response includes the API Processes json application description.

In [17]:
# API Processes - Get Application Details
response, ades_access_token = demo.proc_get_app_details(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

[Request] GET => http://ades.develop.eoepca.org/eric/wps3/processes/s-expression-0_0_2
[App Details] = 200 (OK)
{
  "id": "s-expression-0_0_2",
  "title": "s expressions",
  "description": "Applies s expressions to EO acquisitions",
  "mutable": true,
  "version": "0.0.2",
  "jobControlOptions": [
    "sync-execute",
    "async-execute",
    "dismiss"
  ],
  "outputTransmission": [
    "value",
    "reference"
  ],
  "links": [
    {
      "rel": "http://www.opengis.net/def/rel/ogc/1.0/execute",
      "type": "application/json",
      "title": "Execute End Point",
      "href": "/eric/wps3/processes/s-expression-0_0_2/execution"
    }
  ],
  "inputs": {
    "input_reference": {
      "title": "Input product reference",
      "description": "Input product reference",
      "schema": {
        "type": "string",
        "default": "Any value"
      }
    },
    "s_expression": {
      "title": "s expression",
      "description": "s expression",
      "schema": {
        "type": "string",

#### ADES: Execute Application

**POST {service_url}/processes/{application_name}/execution**<br>
Request body is json as defined by API Processes to define the inputs and outputs, consistent with the CWL Workflow application package.<br>
The response returns **201 CREATED** to indicate that the job has been successfully initiated.<br>
The response **Location header** provides the path (/processes/{application_name}/jobs/{job_id}) to follow the job status.

In [18]:
# API Processes - Execute Application
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, app_name, "../data/s-expression-execute-body.json", id_token=user_id_token, access_token=ades_access_token)

Application execute details read from file: ../data/s-expression-execute-body.json
[Request] POST => http://ades.develop.eoepca.org/eric/wps3/processes/s-expression-0_0_2/execution
[Execute Response] = 201 (Created) => job=/eric/wps3/jobs/7627fd8e-35bb-11ed-ad07-ee207e53c6ea


#### ADES: Job Status

**GET {service_url}/jobs/{job_id}**<br>
Checks the status of the previously submitted job - using the URL returned in the Location header of the execute request.<br>
The response body json provides a status string (success/running/failed) and a % progress.<br>
In the case of a failure then a descriptive message is provided.

In [19]:
# API Processes - Job Status
response, ades_access_token, status = demo.proc_get_job_status(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

[Request] GET => http://ades.develop.eoepca.org/eric/wps3/jobs/7627fd8e-35bb-11ed-ad07-ee207e53c6ea
[Job Status] = 200 (OK) => 16/09/2022 12:31:18 - running => {"jobID":"7627fd8e-35bb-11ed-ad07-ee207e53c6ea","status":"running","message":"ZOO-Kernel is currently running your service!","links":[{"title":"Status location","rel":"status","type":"application\/json","href":"\/eric\/wps3\/jobs\/7627fd8e-35bb-11ed-ad07-ee207e53c6ea"}]}

{
  "jobID": "7627fd8e-35bb-11ed-ad07-ee207e53c6ea",
  "status": "running",
  "message": "ZOO-Kernel is currently running your service!",
  "links": [
    {
      "title": "Status location",
      "rel": "status",
      "type": "application/json",
      "href": "/eric/wps3/jobs/7627fd8e-35bb-11ed-ad07-ee207e53c6ea"
    }
  ]
}


In [20]:
# API Processes - Job Status (keep polling for completion)
response, ades_access_token, status = demo.proc_poll_job_completion(ades_base_url, job_location_path, interval=10, id_token=user_id_token, access_token=ades_access_token)

[Job Status] = 200 (OK) => 16/09/2022 12:31:19 - running => {"jobID":"7627fd8e-35bb-11ed-ad07-ee207e53c6ea","status":"running","message":"ZOO-Kernel is currently running your service!","links":[{"title":"Status location","rel":"status","type":"application\/json","href":"\/eric\/wps3\/jobs\/7627fd8e-35bb-11ed-ad07-ee207e53c6ea"}]}

[Job Status] = 200 (OK) => 16/09/2022 12:31:30 - running => {"jobID":"7627fd8e-35bb-11ed-ad07-ee207e53c6ea","status":"running","message":"ZOO-Kernel is currently running your service!","links":[{"title":"Status location","rel":"status","type":"application\/json","href":"\/eric\/wps3\/jobs\/7627fd8e-35bb-11ed-ad07-ee207e53c6ea"}]}

[Job Status] = 200 (OK) => 16/09/2022 12:31:41 - running => {"jobID":"7627fd8e-35bb-11ed-ad07-ee207e53c6ea","status":"running","message":"ZOO-Kernel is currently running your service!","links":[{"title":"Status location","rel":"status","type":"application\/json","href":"\/eric\/wps3\/jobs\/7627fd8e-35bb-11ed-ad07-ee207e53c6ea"}]}

[

In [21]:
print(response.status_code, response.headers, response.text)

200 {'Date': 'Fri, 16 Sep 2022 12:35:21 GMT', 'Content-Type': 'application/json;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Powered-By': 'ZOO@ZOO-Project', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Set-Cookie': 'auth_rpt-ades=eyJraWQiOiI5OGIzODI0OS1iYjg2LTQ0OGMtOWUxYS00MDhmMGE5ZWIyMGJfc2lnX3JzMjU2IiwidHlwIjoiSldUIiwiYWxnIjoiUlMyNTYifQ.eyJhdWQiOiI2Mzc3NzI4OS01Y2FjLTRiZTEtYjg2MS1hNmZhOGMxNjA1ZTUiLCJwY3RfY2xhaW1zIjp7Ik9TQ0RhdGFPd25lciI6ZmFsc2UsImF1ZCI6ImIwYWUwMzRlLWI3MGItNGY0OS04MWEyLWJiOTkxNDAxNmUxZSIsInN1YiI6ImRlMjg3YTc0LWE1ODEtNDViMi1iNjZiLWIxNGRkYWY0NDE3MyIsInVzZXJfbmFtZSI6ImVyaWMiLCJpc3MiOiJodHRwczovL2F1dGguZGV2ZWxvcC5lb2VwY2Eub3JnIiwiZXhwIjoxNjYzMzM1MDY2LCJpYXQiOjE2NjMzMzE0NjYsIm94T3BlbklEQ29ubmVjdFZlcnNpb24iOiJvcGVuaWRjb25uZWN0LTEuMCIsImlzT3BlcmF0b3IiOnRydWV9LCJwZXJtaXNzaW9ucyI6W3sicmVzb3VyY2Vfc2NvcGVzIjpbInByb3RlY3RlZF9nZXQiLCJvcGVuaWQiXSwicmVzb3VyY2VfaWQiOiI2MmRiZGI5OC1lOTMzLTQ3NDQtODYzZi1mMGQ0ZDAyYmJjNjciLCJleHAiOjE2N

#### ADES: Job Result
**GET {service_url}/jobs/{job_id}/results**<br>
Returns details of the outputs for a successful job execution.<br>
The response body provides json data that includes the reference to the STAC file that indexes the processing outputs.

In [22]:
# API Processes - Job Result
response, ades_access_token, stacCatalogUri = demo.proc_get_job_result(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

[Request] GET => http://ades.develop.eoepca.org/eric/wps3/jobs/7627fd8e-35bb-11ed-ad07-ee207e53c6ea/results
{
  "wf_outputs": {
    "value": {
      "StacCatalogUri": "s3://dd14602fcb6542ba958caab80261b617:develop-user-eric/wf-7627fd8e-35bb-11ed-ad07-ee207e53c6ea/catalog.json"
    }
  }
}


From the S3 url let's retrieve the folder in the bucket where the result files are stored.

In [23]:
result = response.json()["wf_outputs"]["value"]["StacCatalogUri"]
print(f"Result Url: {result}")
result_folder_name = result.rsplit('/', 1)[0].rsplit(':', 1)[1].replace(bucket_name+"/","")
print(f"Result foldername: {result_folder_name}")

Result Url: s3://dd14602fcb6542ba958caab80261b617:develop-user-eric/wf-7627fd8e-35bb-11ed-ad07-ee207e53c6ea/catalog.json
Result foldername: wf-7627fd8e-35bb-11ed-ad07-ee207e53c6ea


#### ADES: List Jobs

In [24]:
# API Processes - List Jobs
response, ades_access_token, job_ids = demo.proc_list_jobs(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print("Jobs:", job_ids)

[Request] GET => http://ades.develop.eoepca.org/eric/wps3/jobs
[Job List] = 200 (OK)
Jobs: ['6cc6d042-35a4-11ed-a2d1-ee207e53c6ea', '1ef71c48-3511-11ed-a20e-3620b8c8d343', '18b2fe48-35ba-11ed-90ca-ee207e53c6ea', '0cd7134c-3511-11ed-8e4c-3620b8c8d343', '28141f5e-350e-11ed-82dc-3620b8c8d343', '4b11cec2-3511-11ed-a270-3620b8c8d343', 'ed41f608-359e-11ed-8066-ee207e53c6ea', '2f864d56-350f-11ed-82b4-3620b8c8d343', 'aa8250ac-3511-11ed-9996-ee207e53c6ea', '82f78c2c-3512-11ed-bd65-ee207e53c6ea', '2d7f4276-3512-11ed-b934-ee207e53c6ea', 'd9228594-350c-11ed-bba6-3620b8c8d343', '726abc42-35a0-11ed-81e8-ee207e53c6ea', '452023c0-3592-11ed-a329-ee207e53c6ea', '7627fd8e-35bb-11ed-ad07-ee207e53c6ea', '0ea9f7be-3513-11ed-8e62-ee207e53c6ea', '11f5cce6-35a3-11ed-b1c5-ee207e53c6ea', '85ced3cc-35a5-11ed-81e6-ee207e53c6ea', 'b752b92a-3516-11ed-80d8-ee207e53c6ea', 'e6105fe8-35a1-11ed-a665-ee207e53c6ea', '9abb827e-359d-11ed-ba9e-ee207e53c6ea', '5ae3ff7a-359b-11ed-b352-ee207e53c6ea', '9f3875ba-35ba-11ed-bfb8-ee2

#### ADES: Undeploy Application
**DELETE {service_url}/processes/{application_name}**<br>
Undeploys application from the ADES

In [25]:
# API Processes - Undeploy Application
response, ades_access_token = demo.proc_undeploy_application(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)

[Request] DELETE => http://ades.develop.eoepca.org/eric/wps3/processes/s-expression-0_0_2
[Undeploy Response] = 204 (No Content)


In [26]:
# Check undeploy - list processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

[Request] GET => http://ades.develop.eoepca.org/eric/wps3/processes
[Process List] = 200 (OK)
{'processes': [{'id': 'UndeployProcess', 'title': 'Undeploy Process', 'description': 'This method will undeploy a deployed processing service.', 'mutable': False, 'version': '1.0.0', 'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'], 'outputTransmission': ['value', 'reference'], 'links': [{'rel': 'self', 'type': 'application/json', 'title': 'Process Description', 'href': '/eric/wps3/processes/UndeployProcess'}]}, {'id': 'GetStatus', 'title': 'Produce an updated ExecuteResponse document. ', 'description': 'Create an ExecuteResponse document from a sid (Service ID), it will use the niternal ZOO Kernel mechanisms to access the current status from a running Service and update the percentCompleted from the original backup file used by the ZOO Kernel when running a Service in background. ', 'mutable': False, 'version': '1.0.0', 'jobControlOptions': ['sync-execute', 'async-execute', '

#### Workspace: Inspect results
Let's inspect the result files in the user S3 bucket.

In [27]:
# Quick hack S3 access
import boto3

# Init S3 session for Creodias
S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)

# List bucket contents
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.filter(Prefix=result_folder_name):
    print(' ->', obj.key)

 -> wf-7627fd8e-35bb-11ed-ad07-ee207e53c6ea/S2B_MSIL2A_20200902T090559_N0214_R050_T34SFH_20200902T113910/S2B_MSIL2A_20200902T090559_N0214_R050_T34SFH_20200902T113910.json
 -> wf-7627fd8e-35bb-11ed-ad07-ee207e53c6ea/S2B_MSIL2A_20200902T090559_N0214_R050_T34SFH_20200902T113910/ndvi.tif
 -> wf-7627fd8e-35bb-11ed-ad07-ee207e53c6ea/catalog.json
